<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/train_sep_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon
!pip install -U ckiptagger[tfgpu,gdown]

Requirement already up-to-date: ckiptagger[gdown,tfgpu] in /usr/local/lib/python3.6/dist-packages (0.1.1)


In [ ]:

import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
print(torch.cuda.is_available())

False


In [ ]:
!cd /content/drive/My Drive/Colab Notebooks/TBrain/nosep

drive  sample_data


In [ ]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [ ]:
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')
        train_data = pd.read_csv(train_path)
        test_data = pd.read_csv(test_path)
        # CONTENT / NAME / CKIPNAME
        train_x = [ self.clean_string(i) for i in train_data['full_content'].values.tolist()]
        train_y = [ self.name_list(i) for i in train_data['name'].values.tolist()]
        train_z = [ self.name_list(i) for i in train_data['ckip_names'].values.tolist() ]

        # CONTENT / NAME
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        self.test_y = [self.name_list(i) for i in test_data['name'].values.tolist()]
        #test_z = [self.name_list(i) for i in test_data['ckip_names'].values.tolist()]

        self.train = [] # CONTENT / NAME / (CKIP-NAME)
        self.test = []  # CONTENT / NAME

        
        qs = 0 
        for i in range(len(train_x)):
          if len(train_y[i]) == 0:
            #self.train.append([train_x[i],[],[]])
            continue
          q = []
          a = []
          for n in train_y[i]:
            qs += 1
            self.train.append([train_x[i],n,1])
          for n in train_z[i]-train_y[i]:
            if len(n)==1: # remove the ckip name len is 1
              continue
            qs += 1
            self.train.append([train_x[i],n,0])              

        for i in range(len(test_x)):
          self.test.append([test_x[i],self.test_y[i],])


        print(f'Train: {len(self.train)} Test: {len(self.test)} Questions: {qs}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        cc = OpenCC('t2s')
        for n in ls:
          res.append(cc.convert(n))
        return set(res)        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      content = self.cc.convert(content)
      return content

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainDataset(Dataset):
    def __init__(self,data,model_type):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
        
    def __getitem__(self,idx):
      question , paragraph ,label = self.data[idx][1] , self.data[idx][0],self.data[idx][2]
      token_tensor = self.tokenizer.encode_plus(question ,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      label_tensor = torch.Tensor([label])
      # token / segment / mask / label
      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor
    def __len__(self):
      return len(self.data)


In [ ]:
class bertwwm(nn.Module):
    def __init__(self,model_name):
        super(bertwwm,self).__init__()
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bi_decoder = nn.Sequential(     
            nn.Linear(768,1)
            ,nn.Sigmoid()
        )
    def forward(self,input_ids):
        all_hidden_states, all_attentions = self.bert_model(input_ids)[-2:]
        binary = self.bi_decoder(all_attentions).reshape(1)
        return binary

In [ ]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
            ,nn.Sigmoid()
        )
        #self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
        print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
def test(model,data,device,ws,pos,ner,model_name):
    #b_modelname = 'bertWWM_BI_16_27_53.pt'
    #bin_model = bertwwm().to(device)
    #bin_model.load_state_dict(torch.load(f'saved_models/{b_modelname}'))
    print('Testing...')
    test_num = len(data.test)
    #criterion = nn.BCELoss()
    #b_tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    c_to_tw = OpenCC('s2t') # chi to tw
    tw_to_c = OpenCC('t2s')
    label_count = 0
    all_predlist = []
    all_labellist = data.test_y
    with torch.no_grad():
      for content,nlabel in data.test:
        if len(nlabel) == 0:
          continue
       # b_input_ids = torch.tensor([b_tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)   
        #print(b_input_ids)
       # pred = bin_model(b_input_ids)
        #print(pred.item())
     #   if pred.item() < 0.4:
      #    all_predlist.append([])
       #   continue
        #if len(nlabel) == 0:
        #  continue
        if len(content)>512:
          content = content[:512] 
        label_count += 1
        ckip_nlist = []
        pred_nlist = []
        ws_results = ws([c_to_tw.convert(content)])
        pos_results = pos(ws_results)
        ner_results = ner(ws_results, pos_results)
        for name in ner_results[0]:
          if name[2] == 'PERSON':
            ckip_nlist.append(tw_to_c.convert(name[3]))
        ckip_nlist = set(ckip_nlist)
        for ckip_name in ckip_nlist:
          tk = tokenizer.encode(str(content),str(ckip_name),max_length=512,truncation=True,pad_to_max_length=True)
         # tk = tokenizer.encode(str(content),str(ckip_name),max_length=512,truncation=True,pad_to_max_length=True)
          input_ids = torch.tensor([tk]).to(device)
          pred = model(input_ids).to('cpu')
          if pred.item()>0.3:
            pred_nlist.append( [ckip_name,pred.item()])
        all_predlist.append(pred_nlist)
        print(f' CKIP {ckip_nlist} PRED {pred_nlist} LABEL {nlabel}')
    return eval(all_predlist,all_labellist)

In [ ]:
def testBIN(model,data,device,ws,pos,ner,model_name):
    print('Testing...')
    acc = 0
    total = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    with torch.no_grad():
      for content,name in data.test:
        input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)   
        b_pred = model(input_ids).to(device) 
        if b_pred.item() < 0.4 and len(name) == 0:
          acc +=1
        elif b_pred.item() > 0.4 and len(name):
          acc +=1
        print(f'C: {content[:20]} Name: {name} Pred: {b_pred.item()}')
    print(f'ALL {total} ACC {acc}')

In [ ]:
"""
config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
model = bertwwmQA(model_type,config)
criterion = nn.BCELoss()

for data in trainLoader:
  token , segment , mask , label = data
  print(type(token),token.shape)
  print(type(segment),segment.shape)
  print(type(mask),mask.shape)
  print(type(label),label.shape)
  pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment)
  print(criterion(pred,label))
  break
"""

'\nconfig = BertConfig.from_pretrained(model_type, output_hidden_states=True)\nmodel = bertwwmQA(model_type,config)\ncriterion = nn.BCELoss()\n\nfor data in trainLoader:\n  token , segment , mask , label = data\n  print(type(token),token.shape)\n  print(type(segment),segment.shape)\n  print(type(mask),mask.shape)\n  print(type(label),label.shape)\n  pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment)\n  print(criterion(pred,label))\n  break\n'

In [ ]:
def train(trainLoader,lr_rate,w_d,device,model_type,epoches):
  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
  model = bertwwmQA(model_type,config).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  criterion = nn.BCELoss()
  for e in range(epoches):
    loss,i = 0,0
    for data in trainLoader:
      i+=1
      token , segment , mask , label = data
      pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment)
      batch_loss = criterion(pred,label)
      loss += batch_loss
      batch_loss.backward()
      optimizer.step()
      optimizer.step()
    print(f'Epoches: {e} Loss {loss} AVG: {loss/i}')
  best_model = copy.deepcopy(model.state_dict())
  return best_model

In [ ]:
def trainBinary(data,lr_rate,w_d,device,model_type):
  model = bertwwm(model_type).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  criterion = nn.BCELoss()
  tokenizer = BertTokenizer.from_pretrained(model_type)
  optimizer.zero_grad()
  loss = 0
  model.train()
  for epoches in range(10):
    print(f'Epoches: {epoches}')
    i = 0
    loss = 0
    for content,question,ans in data.train: 
      i += 1
      input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)
      pred = model(input_ids)
      if len(question)!=0: 
        b_label = torch.Tensor([1]).to(device)
      elif len(question) == 0:
        b_label = torch.Tensor([0]).to(device)
      a_loss = criterion(pred,b_label)
      loss += a_loss
      a_loss.backward()
      del input_ids
      del b_label
      #print(f'i: {i} Content:{content[:10]} Q: {question} Label {b_label} Pred:{pred} Loss:{a_loss}')
      if i % 4 == 0:
        print(f'B LOSS is {loss/4}')
        optimizer.step()
        optimizer.zero_grad()
        loss = 0
  best_model = copy.deepcopy(model.state_dict())
  return best_model

drive/  sample_data/


In [ ]:
data = mydata_nosep('./data0/tbrain_train.csv','./data0/tbrain_test.csv')
train_ds = TrainDataset(mydata.train,model_type)
trainLoader = DataLoader(dds, batch_size=4)

FileNotFoundError: ignored

In [ ]:
!nvidia-smi

In [ ]:
"""
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")
print('CKIP finish')
"""

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
mode = 'train_qa'
model_type = 'hfl/chinese-roberta-wwm-ext'
torch.cuda.empty_cache()  
if mode == 'train_qa':
  model_type = 'hfl/chinese-bert-wwm'
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  print('Train QA')
  best_model = train(trainLoader,0.001,1e-5,device,model_type,1)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_QA_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testqa':
  modelname = 'bertWWM_QA_19_27_44.pt'
  test_model = bertwwmQA(model_type).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  
  test(test_model,data,device,ws,pos,ner,model_type)

# train binary classfier
elif mode == 'trainbin':
  model_type = 'hfl/chinese-bert-wwm'
  best_model = trainBinary(data,0.005,5e-4,device,model_type)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_BIN_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')


elif mode == 'testbin':
  modelname = 'bertWWM_BIN_15_05_01.pt'
  model_type = 'hfl/chinese-bert-wwm'
 # test_model = bertwwm(model_type).to(device)
  test_model = bertwwm(model_type).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  
  testBIN(test_model,data,device,ws,pos,ner,model_type)

In [ ]:
for i in data.test[:10]:
  print(i)

In [ ]:
#RoBERTa-wwm-ext-large	hfl/chinese-roberta-wwm-ext-large
#RoBERTa-wwm-ext	hfl/chinese-roberta-wwm-ext
#BERT-wwm-ext	hfl/chinese-bert-wwm-ext
#BERT-wwm	hfl/chinese-bert-wwm
#RBT3	hfl/rbt3
#RBTL3	hfl/rbtl3